In [1]:
!pip install psycopg2-binary gspread gspread-dataframe pandas oauth2client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.1 MB/s eta 0:00:00


In [2]:
import psycopg2
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default

# Autenticación con tu cuenta de Google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
from google.colab import userdata

# Recuperamos la contraseña guardada de forma segura
db_password = userdata.get('DB_PASSWORD')

In [4]:
# --------------- CONFIGURACIÓN (ajusta estas 4 líneas) ---------------
db_host = 'ec2-52-206-224-222.compute-1.amazonaws.com'
db_name = 'd77mgnf951v3g0'
db_user = 'reports'
sheet_name = 'Prueba automatización COLAB' # <-- El nombre exacto de tu Google Sheet
sql_query = """
    SELECT
      s.terms_accepted_at::date AS fecha_firma,
      COUNT(DISTINCT s.id) AS cantidad_de_firmas
    FROM
      stores s
    WHERE
      s.terms_accepted_at >= CURRENT_DATE - INTERVAL '61 days'
    GROUP BY
      fecha_firma
    ORDER BY
      fecha_firma DESC
"""
# ---------------------------------------------------------------------

print("Iniciando proceso...")

try:
    # 1. Conectar a PostgreSQL
    print("Conectando a la base de datos...")
    conn = psycopg2.connect(
        host=db_host,
        dbname=db_name,
        user=db_user,
        password=db_password,
        port=5432
    )

    # 2. Ejecutar la query y cargarla en un DataFrame de Pandas
    print("Ejecutando consulta...")
    df = pd.read_sql_query(sql_query, conn)
    print(f"Se encontraron {len(df)} filas.")

    # 3. Conectar a Google Sheets y escribir los datos
    print(f"Abriendo Google Sheet: '{sheet_name}'...")
    spreadsheet = gc.open(sheet_name)
    worksheet = spreadsheet.get_worksheet(0) # Obtiene la primera hoja (pestaña)

    print("Limpiando la hoja y escribiendo los nuevos datos...")
    worksheet.clear()
    set_with_dataframe(worksheet, df)

    print("\n¡Proceso completado con éxito! Tu Google Sheet ha sido actualizado.")

except Exception as e:
    print(f"\nERROR: Ha ocurrido un problema.")
    print(e)

finally:
    # 4. Asegurarse de que la conexión siempre se cierre
    if 'conn' in locals() and conn is not None:
        conn.close()
        print("Conexión a la base de datos cerrada.")

Iniciando proceso...
Conectando a la base de datos...
Ejecutando consulta...


/tmp/ipython-input-2894496267.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Se encontraron 60 filas.
Abriendo Google Sheet: 'Prueba automatización COLAB'...
Limpiando la hoja y escribiendo los nuevos datos...

¡Proceso completado con éxito! Tu Google Sheet ha sido actualizado.
Conexión a la base de datos cerrada.
